In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import ParameterGrid


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/My Drive/Colab Notebooks/Merged Research Data By PID - Merged Research Data By PID.csv',index_col=0)

In [ ]:
import pandas as pd

# Assuming your DataFrame is named 'df'
v_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']

for v in v_cols:
    sum_col = f'{v}_sum'
    df[sum_col] = df[[f'{v}-HAMD{i:02d}' for i in range(1, 18)]].sum(axis=1)

df.head()


,UNIQUEID,THERAPY,DRUG,DOSAGE,FREQUENCY,V1-HAMD01,V1-HAMD02,V1-HAMD03,V1-HAMD04,V1-HAMD05,...,V9-HAMD17,V1_sum,V2_sum,V3_sum,V4_sum,V5_sum,V6_sum,V7_sum,V8_sum,V9_sum
0,HMBV-3407,DLX60QD,Duloxetine,60,1,0,0,0,0,0,...,NaN,7,9.0,1.0,2,0.0,0.0,0.0,0.0,0.0
1,HMBV-3319,DLX60QD,Duloxetine,60,1,1,2,0,1,2,...,NaN,14,8.0,8.0,19,0.0,0.0,0.0,0.0,0.0
2,HMBV-1609,DLX60QD,Duloxetine,60,1,3,1,2,1,2,...,NaN,22,9.0,9.0,7,0.0,0.0,0.0,0.0,0.0
3,HMBV-1306,DLX60QD,Duloxetine,60,1,0,1,0,0,2,...,NaN,6,12.0,7.0,11,0.0,0.0,0.0,0.0,0.0
4,HMBV-3403,DLX60QD,Duloxetine,60,1,0,0,0,1,0,...,NaN,8,18.0,7.0,10,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

df['LHAMD'] = 0

# # Iterate over each row
# for index, row in df.iterrows():
#     # Find the last non-zero value in the row
#     last_non_zero = row[row != 0].iloc[-1]
#     # Store the value in a new column 'LastNonZero'
#     df.loc[index, 'LHAMD'] = last_non_zero

# Find the last non-zero value excluding columns starting with 'V1' for each row
last_nonzero_values = df.loc[:, ~df.columns.str.startswith('V1')].apply(lambda row: row[row != 0].iloc[-1], axis=1)

# Create a new column 'Last_Nonzero_Value' and store the values
df['LHAMD'] = last_nonzero_values.fillna(0)


df.loc[df['V1_sum'] < 18, 'FHAMD'] = 'mild'
df.loc[(df['V1_sum'] > 17) & (df['V1_sum'] < 25), 'FHAMD'] = 'moderate'
df.loc[df['V1_sum'] > 24, 'FHAMD'] = 'severe'

df['FHAMD'] = df['V1_sum']
df.drop('V1_sum',axis=1,inplace=True)
df['DHAMD'] = df['FHAMD']- df['LHAMD']

df['FHAMD_level'] = 0
df['Difference'] = 0
df['Remission'] = 0

# FHAMD Groups
df.loc[df['FHAMD'] < 18, 'FHAMD_level'] = 'mild'
df.loc[(df['FHAMD'] > 17) & (df['FHAMD'] < 25), 'FHAMD_level'] = 'moderate'
df.loc[df['FHAMD'] > 24, 'FHAMD_level'] = 'severe'

# Difference
df.loc[df['DHAMD'] < 0, 'Difference'] = 'Worse'
df.loc[df['DHAMD'] == 0, 'Difference'] = 'No change'
df.loc[(df['DHAMD'] >= 1) & (df['DHAMD'] <= 15), 'Difference'] = 'Improvement'
df.loc[df['DHAMD'] >= 16, 'Difference'] = 'Big Improvement'

# Remission
df.loc[df['LHAMD'] <= 7, 'Remission'] = 'Full Remission'
df.loc[(df['LHAMD'] >= 8) & (df['LHAMD'] <= 17), 'Remission'] = 'Remission'
df.loc[(df['LHAMD'] >= 18) & (df['LHAMD'] <= 23), 'Remission'] = 'Partial Remission'
df.loc[df['LHAMD'] >= 24, 'Remission'] = 'Mild Remission'


import pandas as pd

# Custom function to determine label
def determine_label(row):
    # decrease_threshold = 0.5  # 50% decrease threshold
    # increase_threshold = 0.5  # 50% increase threshold

    # if row['FHAMD'] >= row['LHAMD']:
    #     return 'No Response'
    if row['DHAMD'] <= 0:
      return 'No Response'
    else:
      percentage = (row['FHAMD'] - row['LHAMD']) / row['FHAMD']
      if percentage < 0.5:
        return 'Partial Response'
      else:
        return 'Full Response'

    return 0

# Apply the custom function to create the new column 'Response'
df['Response'] = df.apply(determine_label, axis=1)

df

,UNIQUEID,THERAPY,DRUG,DOSAGE,FREQUENCY,V1-HAMD01,V1-HAMD02,V1-HAMD03,V1-HAMD04,V1-HAMD05,...,V7_sum,V8_sum,V9_sum,LHAMD,FHAMD,DHAMD,FHAMD_level,Difference,Remission,Response
0,HMBV-3407,DLX60QD,Duloxetine,60,1,0,0,0,0,0,...,0.0,0.0,0.0,2.0,7,5.0,mild,Improvement,Full Remission,Full Response
1,HMBV-3319,DLX60QD,Duloxetine,60,1,1,2,0,1,2,...,0.0,0.0,0.0,19.0,14,-5.0,mild,Worse,Partial Remission,No Response
2,HMBV-1609,DLX60QD,Duloxetine,60,1,3,1,2,1,2,...,0.0,0.0,0.0,7.0,22,15.0,moderate,Improvement,Full Remission,Full Response
3,HMBV-1306,DLX60QD,Duloxetine,60,1,0,1,0,0,2,...,0.0,0.0,0.0,11.0,6,-5.0,mild,Worse,Remission,No Response
4,HMBV-3403,DLX60QD,Duloxetine,60,1,0,0,0,1,0,...,0.0,0.0,0.0,10.0,8,-2.0,mild,Worse,Remission,No Response
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,HMAQb-30,FLX20QD,Fluoxetine,20,1,3,2,2,2,2,...,14.0,7.0,9.0,9.0,27,18.0,severe,Big Improvement,Remission,Full Response
1464,HMAQb-139,FLX20QD,Fluoxetine,20,1,3,2,0,2,2,...,4.0,8.0,7.0,7.0,26,19.0,severe,Big Improvement,Full Remission,Full Response
1465,HMAQb-39,FLX20QD,Fluoxetine,20,1,2,1,0,1,1,...,19.0,15.0,15.0,15.0,13,-2.0,mild,Worse,Remission,No Response
1466,HMAQb-6,FLX20QD,Fluoxetine,20,1,2,1,0,0,1,...,19.0,13.0,11.0,11.0,20,9.0,moderate,Improvement,Remission,Partial Response


In [ ]:
count_worse = (df['Difference'] == 'Worse').sum()
print('count_worse',count_worse)
count_no_change = (df['Difference'] == 'No change').sum()
print('count_no_change',count_no_change)
count_improvement = (df['Difference'] == 'Improvement').sum()
print('count_improvement',count_improvement)
count_big_improvement = (df['Difference'] == 'Big Improvement').sum()
print('count_big_improvement',count_big_improvement)
count_full_remission = (df['Remission'] == 'Full Remission').sum()
print('count_full_remission',count_full_remission)
count_remission = (df['Remission'] == 'Remission').sum()
print('count_remission',count_remission)
count_partial_remission = (df['Remission'] == 'Partial Remission').sum()
print('count_partial_remission',count_partial_remission)
count_mild_remission = (df['Remission'] == 'Mild Remission').sum()
print('count_mild_remission',count_mild_remission)

count_response = (df['Response'] == 'No Response').sum()
print('count_response',count_response)
count_partial_response = (df['Response'] == 'Partial Response').sum()
print('count_partial_response',count_partial_response)
count_full_response = (df['Response'] == 'Full Response').sum()
print('count_full_response',count_full_response)


count_worse 95
count_no_change 36
count_improvement 1108
count_big_improvement 229
count_full_remission 686
count_remission 641
count_partial_remission 113
count_mild_remission 28
count_response 131
count_partial_response 499
count_full_response 838


In [ ]:

# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])


In [ ]:
df

,UNIQUEID,THERAPY,DRUG,DOSAGE,FREQUENCY,V1-HAMD01,V1-HAMD02,V1-HAMD03,V1-HAMD04,V1-HAMD05,...,V7_sum,V8_sum,V9_sum,LHAMD,FHAMD,DHAMD,FHAMD_level,Difference,Remission,Response
0,768,2,0,60,1,0,0,0,0,0,...,0.0,0.0,0.0,2.0,7,5.0,0,1,0,0
1,764,2,0,60,1,1,2,0,1,2,...,0.0,0.0,0.0,19.0,14,-5.0,0,3,2,1
2,712,2,0,60,1,3,1,2,1,2,...,0.0,0.0,0.0,7.0,22,15.0,1,1,0,0
3,701,2,0,60,1,0,1,0,0,2,...,0.0,0.0,0.0,11.0,6,-5.0,0,3,3,1
4,766,2,0,60,1,0,0,0,1,0,...,0.0,0.0,0.0,10.0,8,-2.0,0,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,31,3,1,20,1,3,2,2,2,2,...,14.0,7.0,9.0,9.0,27,18.0,2,0,3,0
1464,23,3,1,20,1,3,2,0,2,2,...,4.0,8.0,7.0,7.0,26,19.0,2,0,0,0
1465,32,3,1,20,1,2,1,0,1,1,...,19.0,15.0,15.0,15.0,13,-2.0,0,3,3,1
1466,34,3,1,20,1,2,1,0,0,1,...,19.0,13.0,11.0,11.0,20,9.0,1,1,3,2


**SVM**

**Train Data (F1 Score)**

In [ ]:
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
classifier_1 = SVC()
classifier_2 = SVC()
classifier_3 = SVC()
classifier_4 = SVC()

# Train the classifiers independently
classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7887563884156729
Macro Precision: 0.6954225352112676
Weighted Precision: 0.8067639225472083
Micro Recall: 0.7887563884156729
Macro Recall: 0.34194756554307115
Weighted Recall: 0.7887563884156729
Micro F1 Score: 0.7887563884156729
Macro F1 Score: 0.3705491141744819
Weighted F1 Score: 0.7184552817791903
Average F1 Score: 0.6259202614564483

 Remission
Micro Precision: 0.8475298126064736
Macro Precision: 0.8641769747014508
Weighted Precision: 0.8492087404505477
Micro Recall: 0.8475298126064736
Macro Recall: 0.6558247611058787
Weighted Recall: 0.8475298126064736
Micro F1 Score: 0.8475298126064736
Macro F1 Score: 0.712401502147893
Weighted F1 Score: 0.8427451895510129
Average F1 Score: 0.8008921681017932

 Response
Micro Precision: 0.8185689948892675
Macro Precision: 0.856719256028799
Weighted Precision: 0.8257616905789502
Micro Recall: 0.8185689948892675
Macro Recall: 0.6641117541577927
Weighted Recall: 0.8185689948892675
Micro F1 Score: 0.8185689948892675
M

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_train

,FHAMD_level,Difference,Remission,Response
654,1,1,3,0
1338,1,0,0,0
254,1,1,3,2
1070,1,0,0,0
1280,1,1,0,0
...,...,...,...,...
1130,1,1,3,2
1294,1,1,3,0
860,0,1,0,0
1459,0,3,2,1


**Train Accuracy**

In [ ]:
# Calculate accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.9514480408858603
Accuracy: 0.9599659284497445
Accuracy: 0.975298126064736


In [ ]:
y_train['Difference'][:20]

654     1
1338    0
254     1
1070    0
1280    1
889     1
712     2
707     0
86      1
1392    1
265     1
621     1
1452    1
1445    1
1105    1
514     1
705     1
221     1
113     1
1372    0
Name: Difference, dtype: int64

In [ ]:
y_pred['Label_3'][:30]

0     3
1     0
2     3
3     0
4     0
5     0
6     3
7     0
8     3
9     0
10    3
11    0
12    3
13    3
14    3
15    0
16    0
17    2
18    3
19    0
20    0
21    3
22    0
23    3
24    3
25    0
26    0
27    3
28    0
29    0
Name: Label_3, dtype: int64

**F1 Score (Test Data)**

In [ ]:
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
classifier_1 = SVC()
classifier_2 = SVC()
classifier_3 = SVC()
classifier_4 = SVC()

# Train the classifiers independently
classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7721088435374149
Macro Precision: 0.6916376306620209
Weighted Precision: 0.8151058332740763
Micro Recall: 0.7721088435374149
Macro Recall: 0.30710784313725487
Weighted Recall: 0.7721088435374149
Micro F1 Score: 0.7721088435374149
Macro F1 Score: 0.31854355232281656
Weighted F1 Score: 0.6923897782951772
Average F1 Score: 0.5943473913851363

 Remission
Micro Precision: 0.7959183673469388
Macro Precision: 0.8143651786955917
Weighted Precision: 0.8002823824565286
Micro Recall: 0.7959183673469388
Macro Recall: 0.5890383504261159
Weighted Recall: 0.7959183673469388
Micro F1 Score: 0.7959183673469388
Macro F1 Score: 0.6199398574398576
Weighted F1 Score: 0.7894375481110174
Average F1 Score: 0.7350985909659379

 Response
Micro Precision: 0.7891156462585034
Macro Precision: 0.8437282714504241
Weighted Precision: 0.8002449206633007
Micro Recall: 0.7891156462585034
Macro Recall: 0.6001375840908899
Weighted Recall: 0.7891156462585034
Micro F1 Score: 0.789115646258503

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Test Accuracy**

In [ ]:
# Calculate accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.7721088435374149
Accuracy: 0.7959183673469388
Accuracy: 0.7891156462585034


**Logistic Regression**

In [ ]:
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = LogisticRegression()
classifier_3 = LogisticRegression()
classifier_4 = LogisticRegression()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


 Difference
Micro Precision: 0.9063032367972743
Macro Precision: 0.8740401785714286
Weighted Precision: 0.9033273758822097
Micro Recall: 0.9063032367972743
Macro Recall: 0.7119987760998997
Weighted Recall: 0.9063032367972743
Micro F1 Score: 0.9063032367972743
Macro F1 Score: 0.7695557716933714
Weighted F1 Score: 0.900608031880966
Average F1 Score: 0.8588223467905373

 Remission
Micro Precision: 0.8321976149914821
Macro Precision: 0.8175578650584883
Weighted Precision: 0.8302302069708805
Micro Recall: 0.8321976149914821
Macro Recall: 0.7539592679916916
Weighted Recall: 0.8321976149914821
Micro F1 Score: 0.8321976149914821
Macro F1 Score: 0.7805082748758069
Weighted F1 Score: 0.8295512991574465
Average F1 Score: 0.8140857296749119

 Response
Micro Precision: 0.8517887563884157
Macro Precision: 0.8665221368182209
Weighted Precision: 0.8512500042937459
Micro Recall: 0.8517887563884157
Macro Recall: 0.8373667702748234
Weighted Recall: 0.8517887563884157
Micro F1 Score: 0.8517887563884157
M

**Train Accuracy**

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.9063032367972743
Accuracy: 0.8321976149914821
Accuracy: 0.8517887563884157


In [ ]:
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = LogisticRegression()
classifier_3 = LogisticRegression()
classifier_4 = LogisticRegression()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


 Difference
Micro Precision: 0.8027210884353742
Macro Precision: 0.5278846153846154
Weighted Precision: 0.7907014942729229
Micro Recall: 0.8027210884353742
Macro Recall: 0.4910873440285205
Weighted Recall: 0.8027210884353742
Micro F1 Score: 0.802721088435374
Macro F1 Score: 0.5067569805571197
Weighted F1 Score: 0.7945076212062117
Average F1 Score: 0.7013285633995685

 Remission
Micro Precision: 0.7040816326530612
Macro Precision: 0.49444444444444446
Weighted Precision: 0.7014361300075586
Micro Recall: 0.7040816326530612
Macro Recall: 0.4971869131879243
Weighted Recall: 0.7040816326530612
Micro F1 Score: 0.7040816326530612
Macro F1 Score: 0.49351898101898106
Weighted F1 Score: 0.7021371485657201
Average F1 Score: 0.6332459207459208

 Response
Micro Precision: 0.7619047619047619
Macro Precision: 0.6970423797049183
Weighted Precision: 0.7544574799266149
Micro Recall: 0.7619047619047619
Macro Recall: 0.6516058511221966
Weighted Recall: 0.7619047619047619
Micro F1 Score: 0.7619047619047619

**Test Accuracy**

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.8027210884353742
Accuracy: 0.7040816326530612
Accuracy: 0.7619047619047619


**Naive Bayes**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = GaussianNB()
classifier_3 = GaussianNB()
classifier_4 = GaussianNB()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.17206132879045996
Macro Precision: 0.40490430622009566
Weighted Precision: 0.7446895657915115
Micro Recall: 0.17206132879045996
Macro Recall: 0.3361560909735067
Weighted Recall: 0.17206132879045996
Micro F1 Score: 0.17206132879045996
Macro F1 Score: 0.17131953336178685
Weighted F1 Score: 0.07667167148607751
Average F1 Score: 0.14001751121277478

 Remission
Micro Precision: 0.5187393526405452
Macro Precision: 0.4896434346434346
Weighted Precision: 0.6352580948322004
Micro Recall: 0.5187393526405452
Macro Recall: 0.536988463977023
Weighted Recall: 0.5187393526405452
Micro F1 Score: 0.5187393526405452
Macro F1 Score: 0.39701132878899326
Weighted F1 Score: 0.4057704895182632
Average F1 Score: 0.44050705698260056

 Response
Micro Precision: 0.6490630323679727
Macro Precision: 0.5479598956905479
Weighted Precision: 0.6390744758751156
Micro Recall: 0.6490630323679727
Macro Recall: 0.4413433508935971
Weighted Recall: 0.6490630323679727
Micro F1 Score: 0.64906303

**Train Accuracy**

In [ ]:
# Calculate accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.17206132879045996
Accuracy: 0.5187393526405452
Accuracy: 0.6490630323679727


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = GaussianNB()
classifier_3 = GaussianNB()
classifier_4 = GaussianNB()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.18027210884353742
Macro Precision: 0.2107843137254902
Weighted Precision: 0.3238378684807256
Micro Recall: 0.18027210884353742
Macro Recall: 0.33453654188948306
Weighted Recall: 0.18027210884353742
Micro F1 Score: 0.18027210884353745
Macro F1 Score: 0.14556766762649115
Weighted F1 Score: 0.07833770239785277
Average F1 Score: 0.13472582628929378

 Remission
Micro Precision: 0.4965986394557823
Macro Precision: 0.3516154324977855
Weighted Precision: 0.42114485191716083
Micro Recall: 0.4965986394557823
Macro Recall: 0.5179287640233039
Weighted Recall: 0.4965986394557823
Micro F1 Score: 0.4965986394557823
Macro F1 Score: 0.3666061615570917
Weighted F1 Score: 0.38048612067768195
Average F1 Score: 0.41456364056351863

 Response
Micro Precision: 0.6462585034013606
Macro Precision: 0.5393732193732194
Weighted Precision: 0.6307494621780335
Micro Recall: 0.6462585034013606
Macro Recall: 0.45434993627239967
Weighted Recall: 0.6462585034013606
Micro F1 Score: 0.64625

**Test Accuracy**

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.18027210884353742
Accuracy: 0.4965986394557823
Accuracy: 0.6462585034013606


**KNN**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = KNeighborsClassifier()
classifier_3 = KNeighborsClassifier()
classifier_4 = KNeighborsClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.8347529812606473
Macro Precision: 0.6469489120151372
Weighted Precision: 0.8195105558949102
Micro Recall: 0.8347529812606473
Macro Recall: 0.44830828019030267
Weighted Recall: 0.8347529812606473
Micro F1 Score: 0.8347529812606473
Macro F1 Score: 0.5016054609586026
Weighted F1 Score: 0.8023585878219498
Average F1 Score: 0.7129056766803998

 Remission
Micro Precision: 0.9224872231686542
Macro Precision: 0.9168562648500311
Weighted Precision: 0.9231730446680781
Micro Recall: 0.9224872231686542
Macro Recall: 0.7776047119570397
Weighted Recall: 0.9224872231686542
Micro F1 Score: 0.9224872231686542
Macro F1 Score: 0.8186298252499634
Weighted F1 Score: 0.9207429678529233
Average F1 Score: 0.8872866720905136

 Response
Micro Precision: 0.873083475298126
Macro Precision: 0.8572257427581466
Weighted Precision: 0.8735623354489045
Micro Recall: 0.873083475298126
Macro Recall: 0.7593295634675187
Weighted Recall: 0.873083475298126
Micro F1 Score: 0.873083475298126
Mac

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.8347529812606473
Accuracy: 0.9224872231686542
Accuracy: 0.873083475298126


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = KNeighborsClassifier()
classifier_3 = KNeighborsClassifier()
classifier_4 = KNeighborsClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7789115646258503
Macro Precision: 0.5191658391261172
Weighted Precision: 0.752497821372839
Micro Recall: 0.7789115646258503
Macro Recall: 0.3773618538324421
Weighted Recall: 0.7789115646258503
Micro F1 Score: 0.7789115646258504
Macro F1 Score: 0.4093519882179676
Weighted F1 Score: 0.7396541532666085
Average F1 Score: 0.6426392353701421

 Remission
Micro Precision: 0.8639455782312925
Macro Precision: 0.8567516208361279
Weighted Precision: 0.8687179767950105
Micro Recall: 0.8639455782312925
Macro Recall: 0.7128677355674322
Weighted Recall: 0.8639455782312925
Micro F1 Score: 0.8639455782312925
Macro F1 Score: 0.7336155674390968
Weighted F1 Score: 0.8610567459306955
Average F1 Score: 0.8195392972003616

 Response
Micro Precision: 0.8061224489795918
Macro Precision: 0.7318131868131869
Weighted Precision: 0.8002214622112581
Micro Recall: 0.8061224489795918
Macro Recall: 0.6779595350839432
Weighted Recall: 0.8061224489795918
Micro F1 Score: 0.806122448979592
Ma

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.7789115646258503
Accuracy: 0.8639455782312925
Accuracy: 0.8061224489795918


**Random Forests**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = RandomForestClassifier()
classifier_3 = RandomForestClassifier()
classifier_4 = RandomForestClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Remission
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Response
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = RandomForestClassifier()
classifier_3 = RandomForestClassifier()
classifier_4 = RandomForestClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7891156462585034
Macro Precision: 0.4450354609929078
Weighted Precision: 0.7572489988903363
Micro Recall: 0.7891156462585034
Macro Recall: 0.3088235294117647
Weighted Recall: 0.7891156462585034
Micro F1 Score: 0.7891156462585034
Macro F1 Score: 0.3143616012141909
Weighted F1 Score: 0.7219634581208693
Average F1 Score: 0.6084802351978545

 Remission
Micro Precision: 0.8843537414965986
Macro Precision: 0.878453781512605
Weighted Precision: 0.8918673755216372
Micro Recall: 0.8843537414965986
Macro Recall: 0.6616291106938226
Weighted Recall: 0.8843537414965986
Micro F1 Score: 0.8843537414965986
Macro F1 Score: 0.7135133534577707
Weighted F1 Score: 0.8781728945207329
Average F1 Score: 0.8253466631583674

 Response
Micro Precision: 0.7857142857142857
Macro Precision: 0.7329487957903259
Weighted Precision: 0.777299687330913
Micro Recall: 0.7857142857142857
Macro Recall: 0.577131498316072
Weighted Recall: 0.7857142857142857
Micro F1 Score: 0.7857142857142857
Mac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.7891156462585034
Accuracy: 0.8843537414965986
Accuracy: 0.7857142857142857


**Adaboost**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = AdaBoostClassifier()
classifier_3 = AdaBoostClassifier()
classifier_4 = AdaBoostClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7776831345826235
Macro Precision: 0.5590769853927748
Weighted Precision: 0.7754456542988776
Micro Recall: 0.7776831345826235
Macro Recall: 0.5154312062317681
Weighted Recall: 0.7776831345826235
Micro F1 Score: 0.7776831345826235
Macro F1 Score: 0.5295928024191979
Weighted F1 Score: 0.7746974924903398
Average F1 Score: 0.6939911431640539

 Remission
Micro Precision: 0.7819420783645656
Macro Precision: 0.7298966149847932
Weighted Precision: 0.7881288609832049
Micro Recall: 0.7819420783645656
Macro Recall: 0.5967042638904072
Weighted Recall: 0.7819420783645656
Micro F1 Score: 0.7819420783645656
Macro F1 Score: 0.6221392591766022
Weighted F1 Score: 0.761364967896662
Average F1 Score: 0.7218154351459433

 Response
Micro Precision: 0.7904599659284497
Macro Precision: 0.7604241570628223
Weighted Precision: 0.7935891888354718
Micro Recall: 0.7904599659284497
Macro Recall: 0.7169486636256615
Weighted Recall: 0.7904599659284497
Micro F1 Score: 0.7904599659284497
M

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.7776831345826235
Accuracy: 0.7819420783645656
Accuracy: 0.7904599659284497


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = AdaBoostClassifier()
classifier_3 = AdaBoostClassifier()
classifier_4 = AdaBoostClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7346938775510204
Macro Precision: 0.44269943019943025
Weighted Precision: 0.7428610190514952
Micro Recall: 0.7346938775510204
Macro Recall: 0.4190730837789662
Weighted Recall: 0.7346938775510204
Micro F1 Score: 0.7346938775510203
Macro F1 Score: 0.42575081336549236
Weighted F1 Score: 0.7369802101125825
Average F1 Score: 0.6324749670096984

 Remission
Micro Precision: 0.7925170068027211
Macro Precision: 0.8273638841687907
Weighted Precision: 0.8188721927456908
Micro Recall: 0.7925170068027211
Macro Recall: 0.649914536087438
Weighted Recall: 0.7925170068027211
Micro F1 Score: 0.7925170068027211
Macro F1 Score: 0.6930182034982971
Weighted F1 Score: 0.78167575837843
Average F1 Score: 0.755736989559816

 Response
Micro Precision: 0.7755102040816326
Macro Precision: 0.7070939155757193
Weighted Precision: 0.7730591298640166
Micro Recall: 0.7755102040816326
Macro Recall: 0.6688258084120432
Weighted Recall: 0.7755102040816326
Micro F1 Score: 0.7755102040816326
Ma

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.7346938775510204
Accuracy: 0.7925170068027211
Accuracy: 0.7755102040816326


**Decision Tree**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = DecisionTreeClassifier()
classifier_3 = DecisionTreeClassifier()
classifier_4 = DecisionTreeClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Remission
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Response
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = DecisionTreeClassifier()
classifier_3 = DecisionTreeClassifier()
classifier_4 = DecisionTreeClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.7346938775510204
Macro Precision: 0.49855054302422724
Weighted Precision: 0.7329706352047919
Micro Recall: 0.7346938775510204
Macro Recall: 0.47484402852049906
Weighted Recall: 0.7346938775510204
Micro F1 Score: 0.7346938775510203
Macro F1 Score: 0.46260806345412864
Weighted F1 Score: 0.7292950575343048
Average F1 Score: 0.6421989995131513

 Remission
Micro Precision: 0.9693877551020408
Macro Precision: 0.9537953339230429
Weighted Precision: 0.9703266645003866
Micro Recall: 0.9693877551020408
Macro Recall: 0.9513776541961577
Weighted Recall: 0.9693877551020408
Micro F1 Score: 0.9693877551020408
Macro F1 Score: 0.949955022722059
Weighted F1 Score: 0.969220689128513
Average F1 Score: 0.9628544889842042

 Response
Micro Precision: 0.7551020408163265
Macro Precision: 0.6598484039713889
Weighted Precision: 0.759917033964221
Micro Recall: 0.7551020408163265
Macro Recall: 0.6852801904231344
Weighted Recall: 0.7551020408163265
Micro F1 Score: 0.7551020408163265


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.7346938775510204
Accuracy: 0.9693877551020408
Accuracy: 0.7551020408163265


**Gradient Boosting**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = GradientBoostingClassifier()
classifier_3 = GradientBoostingClassifier()
classifier_4 = GradientBoostingClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.9931856899488927
Macro Precision: 0.9977678571428572
Weighted Precision: 0.9932465320029203
Micro Recall: 0.9931856899488927
Macro Recall: 0.9825927817500851
Weighted Recall: 0.9931856899488927
Micro F1 Score: 0.9931856899488927
Macro F1 Score: 0.9900184432730978
Weighted F1 Score: 0.9931346203909478
Average F1 Score: 0.9921129178709794

 Remission
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Response
Micro Precision: 0.9846678023850085
Macro Precision: 0.988327721661055
Weighted Precision: 0.9847217203264903
Micro Recall: 0.9846678023850085
Macro Recall: 0.9746956858663095
Weighted Recall: 0.9846678023850085
Micro F1 Score: 0.9846678023850085
Macro F1 Score: 0.9813178236836738
Weighted F1 Score: 0.9846268889635303
Average F1 Score: 0.9835375050107377


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Example true labels and predicted labels
# true_labels = np.array([0, 1, 0, 1, 0])
# predicted_labels = np.array([0, 1, 1, 1, 0])

# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.9931856899488927
Accuracy: 1.0
Accuracy: 0.9846678023850085


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = GradientBoostingClassifier()
classifier_3 = GradientBoostingClassifier()
classifier_4 = GradientBoostingClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.8333333333333334
Macro Precision: 0.5679862306368331
Weighted Precision: 0.8156439167749717
Micro Recall: 0.8333333333333334
Macro Recall: 0.4557932263814617
Weighted Recall: 0.8333333333333334
Micro F1 Score: 0.8333333333333334
Macro F1 Score: 0.4924852481777161
Weighted F1 Score: 0.8143206605794001
Average F1 Score: 0.7133797473634832

 Remission
Micro Precision: 0.9659863945578231
Macro Precision: 0.9189716621570511
Weighted Precision: 0.9657473760211949
Micro Recall: 0.9659863945578231
Macro Recall: 0.8818870672637104
Weighted Recall: 0.9659863945578231
Micro F1 Score: 0.9659863945578231
Macro F1 Score: 0.8990236632822184
Weighted F1 Score: 0.965508776248281
Average F1 Score: 0.9435062780294409

 Response
Micro Precision: 0.8537414965986394
Macro Precision: 0.7748925142555246
Weighted Precision: 0.8496490400827762
Micro Recall: 0.8537414965986394
Macro Recall: 0.7286588505397854
Weighted Recall: 0.8537414965986394
Micro F1 Score: 0.8537414965986394
M

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.8333333333333334
Accuracy: 0.9659863945578231
Accuracy: 0.8537414965986394


**XGBoost**

In [ ]:
# train
# train
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = XGBClassifier()
classifier_3 = XGBClassifier()
classifier_4 = XGBClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_train)
y_pred_2 = classifier_2.predict(X_train)
y_pred_3 = classifier_3.predict(X_train)
y_pred_4 = classifier_4.predict(X_train)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_train['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_train['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_train['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_train['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_train['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_train['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_train['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_train['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_train['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_train['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_train['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_train['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_train['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_train['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_train['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Remission
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0

 Response
Micro Precision: 1.0
Macro Precision: 1.0
Weighted Precision: 1.0
Micro Recall: 1.0
Macro Recall: 1.0
Weighted Recall: 1.0
Micro F1 Score: 1.0
Macro F1 Score: 1.0
Weighted F1 Score: 1.0
Average F1 Score: 1.0


**Train Accuracy**

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_train['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_train['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0


In [ ]:
# test
# test
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier


# Create a LabelEncoder object
label_encoder = LabelEncoder()
df.fillna(0, inplace=True)

# Iterate over each column in the DataFrame
for column in df.columns:
    # Check if the column data type is object (categorical)
    if df[column].dtype == 'object':
        # Use LabelEncoder to transform the categorical values into numerical labels
        df[column] = label_encoder.fit_transform(df[column])

# Separate features and labels
features = df.drop(['UNIQUEID',	'THERAPY', 'DRUG', 'DOSAGE', 'LHAMD','FHAMD','DHAMD','FREQUENCY', 'FHAMD_level','Difference', 'Remission', 'Response'], axis=1)
labels = df[['FHAMD_level', 'Difference', 'Remission','Response']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create individual SVM classifiers
# classifier_1 = SVC()
classifier_2 = XGBClassifier()
classifier_3 = XGBClassifier()
classifier_4 = XGBClassifier()

# Train the classifiers independently
# classifier_1.fit(X_train, y_train['FHAMD_level'])
classifier_2.fit(X_train, y_train['Difference'])
classifier_3.fit(X_train, y_train['Remission'])
classifier_4.fit(X_train, y_train['Response'])

# Make predictions for each output
# y_pred_1 = classifier_1.predict(X_test)
y_pred_2 = classifier_2.predict(X_test)
y_pred_3 = classifier_3.predict(X_test)
y_pred_4 = classifier_4.predict(X_test)

# Evaluate the performance for each output
# ...

# Perform binary relevance prediction on the test set
y_pred = pd.DataFrame({
    # 'Label_1': y_pred_1,
    'Label_2': y_pred_2,
    'Label_3': y_pred_3,
    'Label_4': y_pred_4
})

from sklearn.metrics import precision_score, recall_score, f1_score

# Difference
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Difference'], y_pred['Label_2'], average='micro')
precision_macro = precision_score(y_test['Difference'], y_pred['Label_2'], average='macro')
precision_weighted = precision_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
recall_micro = recall_score(y_test['Difference'], y_pred['Label_2'], average='micro')
recall_macro = recall_score(y_test['Difference'], y_pred['Label_2'], average='macro')
recall_weighted = recall_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
f1_micro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='micro')
f1_macro_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='macro')
f1_weighted_Difference = f1_score(y_test['Difference'], y_pred['Label_2'], average='weighted')
# Print the results
print('\n Difference')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Difference)
print("Macro F1 Score:", f1_macro_Difference)
print("Weighted F1 Score:", f1_weighted_Difference)
f1_scores = [f1_micro_Difference, f1_macro_Difference, f1_weighted_Difference]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

# Remission
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Remission'], y_pred['Label_3'], average='micro')
precision_macro = precision_score(y_test['Remission'], y_pred['Label_3'], average='macro')
precision_weighted = precision_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
recall_micro = recall_score(y_test['Remission'], y_pred['Label_3'], average='micro')
recall_macro = recall_score(y_test['Remission'], y_pred['Label_3'], average='macro')
recall_weighted = recall_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
f1_micro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='micro')
f1_macro_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='macro')
f1_weighted_Remission = f1_score(y_test['Remission'], y_pred['Label_3'], average='weighted')
# Print the results
print('\n Remission')
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Remission)
print("Macro F1 Score:", f1_macro_Remission)
print("Weighted F1 Score:", f1_weighted_Remission)
f1_scores = [f1_micro_Remission, f1_macro_Remission, f1_weighted_Remission]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)

#Response
# Calculate precision, recall, and F1 score using different average options
precision_micro = precision_score(y_test['Response'], y_pred['Label_4'], average='micro')
precision_macro = precision_score(y_test['Response'], y_pred['Label_4'], average='macro')
precision_weighted = precision_score(y_test['Response'], y_pred['Label_4'], average='weighted')
recall_micro = recall_score(y_test['Response'], y_pred['Label_4'], average='micro')
recall_macro = recall_score(y_test['Response'], y_pred['Label_4'], average='macro')
recall_weighted = recall_score(y_test['Response'], y_pred['Label_4'], average='weighted')
f1_micro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='micro')
f1_macro_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='macro')
f1_weighted_Response = f1_score(y_test['Response'], y_pred['Label_4'], average='weighted')
# Print the results
print("\n Response")
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)
print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)
print("Micro F1 Score:", f1_micro_Response)
print("Macro F1 Score:", f1_macro_Response)
print("Weighted F1 Score:", f1_weighted_Response)
f1_scores = [f1_micro_Response, f1_macro_Response, f1_weighted_Response]
average_f1 = sum(f1_scores) / len(f1_scores)
print("Average F1 Score:", average_f1)



 Difference
Micro Precision: 0.8469387755102041
Macro Precision: 0.6446498875140607
Weighted Precision: 0.843497719672796
Micro Recall: 0.8469387755102041
Macro Recall: 0.45657308377896616
Weighted Recall: 0.8469387755102041
Micro F1 Score: 0.8469387755102041
Macro F1 Score: 0.5065790588514396
Weighted F1 Score: 0.8261108384858359
Average F1 Score: 0.7265428909491599

 Remission
Micro Precision: 0.9795918367346939
Macro Precision: 0.9780860071301247
Weighted Precision: 0.9798510313216197
Micro Recall: 0.9795918367346939
Macro Recall: 0.9444129471808946
Weighted Recall: 0.9795918367346939
Micro F1 Score: 0.9795918367346939
Macro F1 Score: 0.9598303237020831
Weighted F1 Score: 0.9795091005223688
Average F1 Score: 0.9729770869863819

 Response
Micro Precision: 0.8707482993197279
Macro Precision: 0.8192060922228218
Weighted Precision: 0.8692338097318438
Micro Recall: 0.8707482993197279
Macro Recall: 0.7550133785757093
Weighted Recall: 0.8707482993197279
Micro F1 Score: 0.8707482993197279


**Test Accuracy**

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test['Difference'], y_pred['Label_2'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Remission'], y_pred['Label_3'])
print("Accuracy:", accuracy)
accuracy = accuracy_score(y_test['Response'], y_pred['Label_4'])
print("Accuracy:", accuracy)

Accuracy: 0.8469387755102041
Accuracy: 0.9795918367346939
Accuracy: 0.8707482993197279
